# Cargill Ocean Transportation Datathon 2026
## Voyage Optimization & Freight Calculator

---

**Team:** [Your Team Name]

**Objective:** Determine optimal vessel-cargo assignments for Cargill's Capesize fleet and committed cargoes to maximize portfolio profit.

---

## 1. Setup & Imports

In [84]:
import os
import sys

# Ensure we're in the correct directory
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
if notebook_dir not in sys.path:
    sys.path.insert(0, notebook_dir)

# Clear any cached imports
for mod in list(sys.modules.keys()):
    if 'portfolio_optimizer' in mod or 'freight_calculator' in mod:
        del sys.modules[mod]

import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend (must be before pyplot import)
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Our custom modules
from freight_calculator import (
    FreightCalculator, PortDistanceManager, BunkerPrices,
    Vessel, Cargo, VoyageResult,
    create_cargill_vessels, create_cargill_cargoes,
    create_market_vessels, create_market_cargoes, create_bunker_prices
)
from portfolio_optimizer import (
    PortfolioOptimizer, ScenarioAnalyzer, FullPortfolioOptimizer
)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('seaborn-v0_8-whitegrid')

print("Setup complete!")

Setup complete!


## 2. Initialize Calculator & Load Data

In [85]:
# Initialize the freight calculator
distance_mgr = PortDistanceManager('Port_Distances.csv')
bunker_prices = create_bunker_prices()
calculator = FreightCalculator(distance_mgr, bunker_prices)
optimizer = PortfolioOptimizer(calculator)
analyzer = ScenarioAnalyzer(optimizer)
full_optimizer = FullPortfolioOptimizer(calculator)

# Load vessel and cargo data
cargill_vessels = create_cargill_vessels()
cargill_cargoes = create_cargill_cargoes()
market_vessels = create_market_vessels()
market_cargoes = create_market_cargoes()

print(f"Loaded {len(cargill_vessels)} Cargill vessels")
print(f"Loaded {len(cargill_cargoes)} Cargill committed cargoes")
print(f"Loaded {len(market_vessels)} market vessels")
print(f"Loaded {len(market_cargoes)} market cargoes available for bidding")

Loaded 4 Cargill vessels
Loaded 3 Cargill committed cargoes
Loaded 11 market vessels
Loaded 8 market cargoes available for bidding


## 3. Fleet Overview

In [86]:
# Display Cargill vessels
vessel_data = []
for v in cargill_vessels:
    vessel_data.append({
        'Vessel': v.name,
        'DWT (MT)': f"{v.dwt:,}",
        'Hire Rate ($/day)': f"${v.hire_rate:,}",
        'Current Port': v.current_port,
        'ETD': v.etd,
        'VLSFO ROB (MT)': v.bunker_rob_vlsfo,
        'Speed Laden (kn)': v.speed_laden,
        'Speed Ballast (kn)': v.speed_ballast,
    })

print("\n[SHIP] CARGILL VESSELS")
print("=" * 80)
pd.DataFrame(vessel_data)


[SHIP] CARGILL VESSELS


,Vessel,DWT (MT),Hire Rate ($/day),Current Port,ETD,VLSFO ROB (MT),Speed Laden (kn),Speed Ballast (kn)
0,ANN BELL,"180,803","$11,750",QINGDAO,25 Feb 2026,401.3,13.5,14.5
1,OCEAN HORIZON,"181,550","$15,750",MAP TA PHUT,1 Mar 2026,265.8,13.8,14.8
2,PACIFIC GLORY,"182,320","$14,800",GWANGYANG,10 Mar 2026,601.9,13.5,14.2
3,GOLDEN ASCENT,"179,965","$13,950",FANGCHENG,8 Mar 2026,793.3,13.0,14.0


In [87]:
# Display Cargill cargoes
cargo_data = []
for c in cargill_cargoes:
    cargo_data.append({
        'Cargo': c.name[:35],
        'Customer': c.customer,
        'Commodity': c.commodity,
        'Quantity (MT)': f"{c.quantity:,}",
        'Laycan': f"{c.laycan_start} - {c.laycan_end}",
        'Freight ($/MT)': f"${c.freight_rate:.2f}",
        'Load Port': c.load_port[:20],
        'Discharge Port': c.discharge_port,
    })

print("\n[PKG] CARGILL COMMITTED CARGOES")
print("=" * 80)
pd.DataFrame(cargo_data)


[PKG] CARGILL COMMITTED CARGOES


,Cargo,Customer,Commodity,Quantity (MT),Laycan,Freight ($/MT),Load Port,Discharge Port
0,EGA Bauxite (Guinea-China),EGA,Bauxite,"180,000",2 Apr 2026 - 10 Apr 2026,$23.00,KAMSAR ANCHORAGE,QINGDAO
1,BHP Iron Ore (Australia-China),BHP,Iron Ore,"160,000",7 Mar 2026 - 11 Mar 2026,$9.00,PORT HEDLAND,LIANYUNGANG
2,CSN Iron Ore (Brazil-China),CSN,Iron Ore,"180,000",1 Apr 2026 - 8 Apr 2026,$22.30,ITAGUAI,QINGDAO


## 3.5 Vessel Feasibility Matrix

Before optimizing, we need to understand which vessels can physically make which cargo laycans based on their current position and ETD.

In [88]:
# Build complete feasibility matrix dynamically
feasibility_results = {}

print("VESSEL FEASIBILITY ANALYSIS")
print("=" * 70)

for vessel in cargill_vessels:
    print(f"\n{vessel.name} (at {vessel.current_port}, ETD {vessel.etd}):")
    
    for cargo in cargill_cargoes:
        try:
            result = calculator.calculate_voyage(vessel, cargo, use_eco_speed=True)
            feasibility_results[(vessel.name, cargo.name)] = {
                'can_make': result.can_make_laycan,
                'profit': result.net_profit if result.can_make_laycan else None,
                'tce': result.tce if result.can_make_laycan else None,
                'arrival': result.arrival_date,
                'laycan_end': result.laycan_end,
                'result': result
            }
            
            cargo_short = cargo.name.split('(')[0].strip()
            if result.can_make_laycan:
                print(f"  {cargo_short:20} YES  Profit: ${result.net_profit:>10,.0f}  TCE: ${result.tce:>8,.0f}/day")
            else:
                arr = result.arrival_date.strftime("%d %b")
                end = result.laycan_end.strftime("%d %b")
                margin = (result.laycan_end - result.arrival_date).total_seconds() / 86400
                print(f"  {cargo_short:20} NO   (arrives {arr}, laycan ends {end}, {margin:+.1f} days)")
        except Exception as e:
            print(f"  {cargo.name[:20]:20} ERR  {e}")
            feasibility_results[(vessel.name, cargo.name)] = {'can_make': False, 'error': str(e)}

VESSEL FEASIBILITY ANALYSIS

ANN BELL (at QINGDAO, ETD 25 Feb 2026):
  EGA Bauxite          YES  Profit: $ 1,407,338  TCE: $  27,438/day
  BHP Iron Ore         YES  Profit: $   221,198  TCE: $  18,349/day
  CSN Iron Ore         YES  Profit: $ 1,018,508  TCE: $  23,390/day

OCEAN HORIZON (at MAP TA PHUT, ETD 1 Mar 2026):
  EGA Bauxite          YES  Profit: $ 1,264,492  TCE: $  30,850/day
  BHP Iron Ore         YES  Profit: $   178,078  TCE: $  21,552/day
  CSN Iron Ore         NO   (arrives 10 Apr, laycan ends 08 Apr, -2.7 days)

PACIFIC GLORY (at GWANGYANG, ETD 10 Mar 2026):
  EGA Bauxite          NO   (arrives 16 Apr, laycan ends 10 Apr, -6.7 days)
  BHP Iron Ore         NO   (arrives 21 Mar, laycan ends 11 Mar, -10.4 days)
  CSN Iron Ore         NO   (arrives 17 Apr, laycan ends 08 Apr, -9.7 days)

GOLDEN ASCENT (at FANGCHENG, ETD 8 Mar 2026):
  EGA Bauxite          NO   (arrives 11 Apr, laycan ends 10 Apr, -1.3 days)
  BHP Iron Ore         NO   (arrives 17 Mar, laycan ends 11 Mar, -

In [89]:
# Build dynamic feasibility summary
print("\nVESSEL FEASIBILITY SUMMARY")
print("=" * 60)

vessels_with_options = 0
vessel_cargo_options = {}  # Store for later use

for vessel in cargill_vessels:
    feasible_cargoes = []
    for cargo in cargill_cargoes:
        if feasibility_results.get((vessel.name, cargo.name), {}).get('can_make', False):
            feasible_cargoes.append(cargo.name.split('(')[0].strip())
    
    vessel_cargo_options[vessel.name] = feasible_cargoes
    
    if feasible_cargoes:
        vessels_with_options += 1
        print(f"  {vessel.name:18} Can make: {', '.join(feasible_cargoes)}")
    else:
        print(f"  {vessel.name:18} Cannot make any cargo laycans")

print(f"\n  Total: {vessels_with_options} of {len(cargill_vessels)} vessels can make at least one cargo")

# Highlight vessels with most flexibility
max_options = max(len(opts) for opts in vessel_cargo_options.values())
most_flexible = [v for v, opts in vessel_cargo_options.items() if len(opts) == max_options and len(opts) > 0]
if most_flexible:
    print(f"\n  Most flexible vessel(s): {', '.join(most_flexible)} ({max_options} cargo options each)")


VESSEL FEASIBILITY SUMMARY
  ANN BELL           Can make: EGA Bauxite, BHP Iron Ore, CSN Iron Ore
  OCEAN HORIZON      Can make: EGA Bauxite, BHP Iron Ore
  PACIFIC GLORY      Cannot make any cargo laycans
  GOLDEN ASCENT      Cannot make any cargo laycans

  Total: 2 of 4 vessels can make at least one cargo

  Most flexible vessel(s): ANN BELL (3 cargo options each)


## 4. Voyage Analysis - All Combinations

In [90]:
# Calculate all voyage combinations
all_voyages = optimizer.calculate_all_voyages(cargill_vessels, cargill_cargoes)

# Create summary view
summary = all_voyages[['vessel', 'cargo', 'can_make_laycan', 'total_days', 
                       'net_freight', 'total_bunker_cost', 'net_profit', 'tce']].copy()
summary['cargo'] = summary['cargo'].str[:30]
summary['net_freight'] = summary['net_freight'].apply(lambda x: f"${x:,.0f}" if pd.notna(x) else 'N/A')
summary['total_bunker_cost'] = summary['total_bunker_cost'].apply(lambda x: f"${x:,.0f}" if pd.notna(x) else 'N/A')
summary['net_profit'] = summary['net_profit'].apply(lambda x: f"${x:,.0f}" if pd.notna(x) else 'N/A')
summary['tce'] = summary['tce'].apply(lambda x: f"${x:,.0f}" if pd.notna(x) else 'N/A')

print("\n[CHART] ALL VOYAGE COMBINATIONS")
print("=" * 100)
summary


[CHART] ALL VOYAGE COMBINATIONS


,vessel,cargo,can_make_laycan,total_days,net_freight,total_bunker_cost,net_profit,tce
0,ANN BELL,EGA Bauxite (Guinea-China),True,89.71,"$4,026,994","$1,550,597","$1,407,338","$27,438"
1,ANN BELL,BHP Iron Ore (Australia-China),True,33.52,"$1,524,600","$514,548","$221,198","$18,349"
2,ANN BELL,CSN Iron Ore (Brazil-China),True,87.50,"$3,805,587","$1,578,944","$1,018,508","$23,390"
3,OCEAN HORIZON,EGA Bauxite (Guinea-China),True,83.74,"$4,043,961","$1,445,564","$1,264,492","$30,850"
4,OCEAN HORIZON,BHP Iron Ore (Australia-China),True,30.69,"$1,524,600","$468,125","$178,078","$21,552"
5,OCEAN HORIZON,CSN Iron Ore (Brazil-China),False,89.36,"$3,821,621","$1,650,341","$583,826","$22,283"
6,PACIFIC GLORY,EGA Bauxite (Guinea-China),False,89.83,"$4,061,449","$1,616,938","$1,099,954","$27,044"
7,PACIFIC GLORY,BHP Iron Ore (Australia-China),False,33.07,"$1,524,600","$526,736","$113,408","$18,229"
8,PACIFIC GLORY,CSN Iron Ore (Brazil-China),False,87.74,"$3,838,148","$1,649,658","$709,936","$22,891"
9,GOLDEN ASCENT,EGA Bauxite (Guinea-China),False,87.53,"$4,007,961","$1,476,742","$1,295,137","$28,746"


In [91]:
# TCE Heatmap
pivot_tce = all_voyages.pivot_table(
    index='vessel',
    columns='cargo',
    values='tce',
    aggfunc='first'
)
pivot_tce.columns = [c[:20] for c in pivot_tce.columns]

plt.figure(figsize=(12, 6))
sns.heatmap(pivot_tce, annot=True, fmt=',.0f', cmap='RdYlGn', 
            cbar_kws={'label': 'TCE ($/day)'})
plt.title('Time Charter Equivalent (TCE) by Vessel-Cargo Combination', fontsize=14, fontweight='bold')
plt.xlabel('Cargo')
plt.ylabel('Vessel')
plt.tight_layout()
plt.savefig('tce_heatmap.png', dpi=150)
plt.show()

In [92]:
# Laycan Feasibility Matrix
pivot_laycan = all_voyages.pivot_table(
    index='vessel',
    columns='cargo',
    values='can_make_laycan',
    aggfunc='first'
)
pivot_laycan.columns = [c[:20] for c in pivot_laycan.columns]

plt.figure(figsize=(12, 6))
sns.heatmap(pivot_laycan.astype(int), annot=pivot_laycan.replace({True: '[OK]', False: '[X]'}), 
            fmt='', cmap=['#ffcccc', '#ccffcc'], cbar=False)
plt.title('Laycan Feasibility Matrix', fontsize=14, fontweight='bold')
plt.xlabel('Cargo')
plt.ylabel('Vessel')
plt.tight_layout()
plt.savefig('laycan_matrix.png', dpi=150)
plt.show()

## 5. Portfolio Optimization

In [93]:
# Optimize vessel-cargo assignments
portfolio = optimizer.optimize_assignments(
    cargill_vessels, 
    cargill_cargoes,
    maximize='profit'
)

print("\n" + "=" * 80)
print("[TROPHY] OPTIMAL PORTFOLIO ASSIGNMENTS")
print("=" * 80)

for vessel, cargo, result in portfolio.assignments:
    print(f"\n[OK] {vessel} -> {cargo}")
    if result:
        print(f"   [DATE] Arrival: {result.arrival_date.strftime('%d %b %Y')} | Laycan ends: {result.laycan_end.strftime('%d %b %Y')}")
        print(f"   [TIME]  Duration: {result.total_days:.1f} days")
        print(f"   [PKG] Cargo: {result.cargo_quantity:,} MT")
        print(f"   [MONEY] TCE: ${result.tce:,.0f}/day")
        print(f"   [UP] Net Profit: ${result.net_profit:,.0f}")

print(f"\n" + "-" * 50)
print(f"[MONEY] TOTAL PORTFOLIO PROFIT: ${portfolio.total_profit:,.0f}")
print(f"[CHART] AVERAGE TCE: ${portfolio.avg_tce:,.0f}/day")


[TROPHY] OPTIMAL PORTFOLIO ASSIGNMENTS

[OK] ANN BELL -> CSN Iron Ore (Brazil-China)
   [DATE] Arrival: 03 Apr 2026 | Laycan ends: 08 Apr 2026
   [TIME]  Duration: 87.5 days
   [PKG] Cargo: 177,303 MT
   [MONEY] TCE: $23,390/day
   [UP] Net Profit: $1,018,508

[OK] OCEAN HORIZON -> EGA Bauxite (Guinea-China)
   [DATE] Arrival: 31 Mar 2026 | Laycan ends: 10 Apr 2026
   [TIME]  Duration: 83.7 days
   [PKG] Cargo: 178,050 MT
   [MONEY] TCE: $30,850/day
   [UP] Net Profit: $1,264,492

--------------------------------------------------
[MONEY] TOTAL PORTFOLIO PROFIT: $2,283,001
[CHART] AVERAGE TCE: $27,120/day


In [94]:
# Show unassigned items
if portfolio.unassigned_cargoes:
    print("\n[!]  UNASSIGNED CARGOES (Need market vessels):")
    for c in portfolio.unassigned_cargoes:
        print(f"   * {c}")

if portfolio.unassigned_vessels:
    print("\n[LIST] AVAILABLE VESSELS (For market cargoes):")
    for v in portfolio.unassigned_vessels:
        print(f"   * {v}")


[!]  UNASSIGNED CARGOES (Need market vessels):
   * BHP Iron Ore (Australia-China)

[LIST] AVAILABLE VESSELS (For market cargoes):
   * PACIFIC GLORY
   * GOLDEN ASCENT


## 6. Scenario Analysis

In [95]:
# Find tipping points
tipping_points = analyzer.find_tipping_points(cargill_vessels, cargill_cargoes)

# Get search limits from the result
max_bunker = tipping_points.get('max_bunker_searched_pct', 100)
max_delay = tipping_points.get('max_delay_searched_days', 20)

print("\n" + "=" * 80)
print("TIPPING POINT ANALYSIS")
print("=" * 80)

if tipping_points['bunker']:
    bp = tipping_points['bunker']
    bunker_tipping_pct = bp['change_pct']
    print(f"\nBUNKER PRICE TIPPING POINT:")
    print(f"   If bunker prices increase by more than {bunker_tipping_pct:.0f}%,")
    print(f"   the optimal assignment strategy changes.")
else:
    bunker_tipping_pct = None
    print(f"\nBUNKER PRICE: No tipping point found up to +{max_bunker:.0f}% increase")
    print(f"   Current recommendation is robust to bunker price volatility.")

if tipping_points['port_delay']:
    pd_tp = tipping_points['port_delay']
    delay_tipping_days = pd_tp['days']
    print(f"\nPORT DELAY TIPPING POINT:")
    print(f"   If port delays exceed {delay_tipping_days} days,")
    print(f"   the optimal assignment strategy changes.")
else:
    delay_tipping_days = None
    print(f"\nPORT DELAY: No tipping point found up to {max_delay} days")
    print(f"   Current recommendation is robust to port congestion.")


TIPPING POINT ANALYSIS

BUNKER PRICE TIPPING POINT:
   If bunker prices increase by more than 60%,
   the optimal assignment strategy changes.

PORT DELAY: No tipping point found up to 20 days
   Current recommendation is robust to port congestion.


In [96]:
# Bunker price sensitivity chart
bunker_analysis = analyzer.analyze_bunker_sensitivity(
    cargill_vessels, cargill_cargoes,
    price_range=(0.8, 1.4),
    steps=13
)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Profit chart
ax1.plot(bunker_analysis['bunker_change_pct'], bunker_analysis['total_profit'] / 1e6, 
         marker='o', linewidth=2, color='#2ecc71')
ax1.axvline(x=0, color='red', linestyle='--', alpha=0.5, label='Baseline')
ax1.set_xlabel('Bunker Price Change (%)', fontsize=12)
ax1.set_ylabel('Total Profit ($ Million)', fontsize=12)
ax1.set_title('Portfolio Profit vs Bunker Price', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# TCE chart
ax2.plot(bunker_analysis['bunker_change_pct'], bunker_analysis['avg_tce'], 
         marker='s', linewidth=2, color='#3498db')
ax2.axvline(x=0, color='red', linestyle='--', alpha=0.5, label='Baseline')
ax2.set_xlabel('Bunker Price Change (%)', fontsize=12)
ax2.set_ylabel('Average TCE ($/day)', fontsize=12)
ax2.set_title('Average TCE vs Bunker Price', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('bunker_sensitivity.png', dpi=150)
plt.show()

In [97]:
# Port delay sensitivity chart
delay_analysis = analyzer.analyze_port_delay_sensitivity(
    cargill_vessels, cargill_cargoes,
    max_delay_days=14
)

fig, ax = plt.subplots(figsize=(10, 5))

ax.fill_between(delay_analysis['port_delay_days'], 
                delay_analysis['total_profit'] / 1e6,
                alpha=0.3, color='#e74c3c')
ax.plot(delay_analysis['port_delay_days'], delay_analysis['total_profit'] / 1e6, 
        marker='o', linewidth=2, color='#e74c3c')
ax.set_xlabel('Additional Port Delay (Days)', fontsize=12)
ax.set_ylabel('Total Profit ($ Million)', fontsize=12)
ax.set_title('Portfolio Profit vs Port Delay (China Ports)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

# Add cost per day annotation
profit_loss_per_day = (delay_analysis['total_profit'].iloc[0] - delay_analysis['total_profit'].iloc[-1]) / 14
ax.annotate(f'Cost: ~${profit_loss_per_day/1000:,.0f}K per delay day', 
            xy=(7, delay_analysis['total_profit'].mean() / 1e6),
            fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig('delay_sensitivity.png', dpi=150)
plt.show()

## 7. Market Vessel Analysis (For Unassigned BHP Cargo)

In [98]:
# Since BHP cargo is unassigned, analyze market vessels that could carry it
bhp_cargo = [c for c in cargill_cargoes if 'BHP' in c.name][0]

print("\n" + "=" * 80)
print("[SHIP] MARKET VESSEL ANALYSIS FOR BHP IRON ORE CARGO")
print("=" * 80)
print(f"\nCargo: {bhp_cargo.name}")
print(f"Laycan: {bhp_cargo.laycan_start} - {bhp_cargo.laycan_end}")
print(f"Load Port: {bhp_cargo.load_port}")

# Calculate voyage for market vessels
market_results = []
for vessel in market_vessels:
    try:
        result = calculator.calculate_voyage(vessel, bhp_cargo, use_eco_speed=True)
        market_results.append({
            'Vessel': vessel.name,
            'Current Port': vessel.current_port,
            'ETD': vessel.etd,
            'Can Make Laycan': '[OK]' if result.can_make_laycan else '[X]',
            'Arrival': result.arrival_date.strftime('%d %b'),
            'Voyage Days': f"{result.total_days:.1f}",
            'TCE ($/day)': f"${result.tce:,.0f}",
            'result': result
        })
    except Exception as e:
        market_results.append({
            'Vessel': vessel.name,
            'Current Port': vessel.current_port,
            'ETD': vessel.etd,
            'Can Make Laycan': '[X]',
            'Error': str(e)[:30]
        })

market_df = pd.DataFrame(market_results)
display_cols = [c for c in market_df.columns if c != 'result']
market_df[display_cols]


[SHIP] MARKET VESSEL ANALYSIS FOR BHP IRON ORE CARGO

Cargo: BHP Iron Ore (Australia-China)
Laycan: 7 Mar 2026 - 11 Mar 2026
Load Port: PORT HEDLAND


,Vessel,Current Port,ETD,Can Make Laycan,Arrival,Voyage Days,TCE ($/day)
0,ATLANTIC FORTUNE,PARADIP,2 Mar 2026,[X],11 Mar,31.2,"$20,791"
1,PACIFIC VANGUARD,CAOFEIDIAN,26 Feb 2026,[OK],10 Mar,34.5,"$17,304"
2,CORAL EMPEROR,ROTTERDAM,5 Mar 2026,[X],12 Apr,60.9,"$1,791"
3,EVEREST OCEAN,XIAMEN,3 Mar 2026,[X],12 Mar,30.6,"$21,664"
4,POLARIS SPIRIT,KANDLA,28 Feb 2026,[X],15 Mar,37.1,"$14,001"
5,IRON CENTURY,PORT TALBOT,9 Mar 2026,[X],16 Apr,60.2,"$1,573"
6,MOUNTAIN TRADER,GWANGYANG,6 Mar 2026,[X],17 Mar,33.3,"$18,640"
7,NAVIS PRIDE,MUNDRA,27 Feb 2026,[X],14 Mar,36.7,"$14,668"
8,AURORA SKY,JINGTANG,4 Mar 2026,[X],15 Mar,33.8,"$18,255"
9,ZENITH GLORY,VIZAG,7 Mar 2026,[X],20 Mar,35.0,"$16,303"


In [99]:
# Find best market vessel for BHP cargo
valid_market = [r for r in market_results if r.get('Can Make Laycan') == '[OK]' and 'result' in r]

if valid_market:
    best = max(valid_market, key=lambda x: x['result'].tce)
    print(f"\n[TROPHY] RECOMMENDED MARKET VESSEL FOR BHP CARGO:")
    print(f"   Vessel: {best['Vessel']}")
    print(f"   TCE: {best['TCE ($/day)']}")
    print(f"   Arrival: {best['Arrival']} (Laycan ends: {bhp_cargo.laycan_end})")
    bhp_market_vessel = best['Vessel']
    bhp_market_count = len(valid_market)
else:
    print("\n[!] No market vessels can make the BHP cargo laycan!")
    bhp_market_vessel = None
    bhp_market_count = 0


[TROPHY] RECOMMENDED MARKET VESSEL FOR BHP CARGO:
   Vessel: PACIFIC VANGUARD
   TCE: $17,304
   Arrival: 10 Mar (Laycan ends: 11 Mar 2026)


## 8. Full Portfolio Optimization (Joint Optimization)

**Key Innovation:** Instead of a greedy 3-step approach, we use **joint optimization** that considers all valid combinations simultaneously:

1. **Cargill vessels → Cargill cargoes** (committed obligations)
2. **Cargill vessels → Market cargoes** (opportunity to earn more on lucrative routes)
3. **Market vessels → Cargill cargoes** (hire at FFA rate: $18,000/day to cover commitments)

**Why this matters:** A Cargill vessel might earn MORE on a market cargo than a Cargill cargo. We can hire a market vessel cheaply for the Cargill cargo and come out ahead overall.

Note: Market vessels on market cargoes is NOT a valid combination for Cargill.

In [100]:
# Run full portfolio optimization
full_result = full_optimizer.optimize_full_portfolio(
    cargill_vessels=cargill_vessels,
    market_vessels=market_vessels,
    cargill_cargoes=cargill_cargoes,
    market_cargoes=market_cargoes,
    target_tce=18000,  # Target TCE for market cargo bids
)

print("=" * 80)
print("FULL PORTFOLIO OPTIMIZATION RESULTS")
print("=" * 80)

print("\n--- CARGILL VESSEL ASSIGNMENTS ---")
for vessel_name, cargo_name, option in full_result.cargill_vessel_assignments:
    cargo_type = "Cargill" if any(cargo_name == c.name for c in cargill_cargoes) else "Market"
    print(f"\n  {vessel_name} -> {cargo_name[:40]}")
    print(f"    Type: {cargo_type} cargo")
    print(f"    Profit: ${option.net_profit:,.0f}")
    print(f"    TCE: ${option.tce:,.0f}/day")
    if cargo_type == "Market":
        print(f"    Min Freight Bid: ${option.min_freight_bid:.2f}/MT")

print("\n--- MARKET VESSEL ASSIGNMENTS (for Cargill cargoes) ---")
if full_result.market_vessel_assignments:
    for vessel_name, cargo_name, option in full_result.market_vessel_assignments:
        print(f"\n  {vessel_name} (Market) -> {cargo_name[:40]}")
        print(f"    Recommended Max Hire: ${option.recommended_hire_rate:,.0f}/day")
        print(f"    Voyage Duration: {option.voyage_days:.1f} days")
else:
    print("  None required")

print("\n--- UNASSIGNED ---")
print(f"  Cargill cargoes without coverage: {full_result.unassigned_cargill_cargoes or 'None'}")
print(f"  Cargill vessels without cargo: {full_result.unassigned_cargill_vessels or 'None'}")

print("\n" + "=" * 80)
print(f"TOTAL PORTFOLIO PROFIT: ${full_result.total_profit:,.0f}")
print("=" * 80)

FULL PORTFOLIO OPTIMIZATION RESULTS

--- CARGILL VESSEL ASSIGNMENTS ---

  OCEAN HORIZON -> BHP Iron Ore (Australia-China)
    Type: Cargill cargo
    Profit: $178,078
    TCE: $21,552/day

  ANN BELL -> Adaro Coal (Indonesia-India)
    Type: Market cargo
    Profit: $1,146,211
    TCE: $29,333/day
    Min Freight Bid: $15.17/MT

  PACIFIC GLORY -> Teck Coking Coal (Canada-China)
    Type: Market cargo
    Profit: $735,206
    TCE: $30,189/day
    Min Freight Bid: $15.74/MT

  GOLDEN ASCENT -> Rio Tinto Iron Ore (Australia-China)
    Type: Market cargo
    Profit: $407,376
    TCE: $27,336/day
    Min Freight Bid: $9.83/MT

--- MARKET VESSEL ASSIGNMENTS (for Cargill cargoes) ---

  IRON CENTURY (Market) -> EGA Bauxite (Guinea-China)
    Recommended Max Hire: $21,451/day
    Voyage Duration: 76.7 days

  CORAL EMPEROR (Market) -> CSN Iron Ore (Brazil-China)
    Recommended Max Hire: $13,892/day
    Voyage Duration: 76.9 days

--- UNASSIGNED ---
  Cargill cargoes without coverage: None
 

In [101]:
# Show JOINT OPTIMIZATION decision analysis
print("=" * 80)
print("JOINT OPTIMIZATION BREAKDOWN")
print("=" * 80)

print("\n📊 OPTIMIZATION APPROACH:")
print("  The joint optimizer considers ALL valid combinations simultaneously:")
print("    1. Cargill vessels → Cargill cargoes (committed obligations)")
print("    2. Cargill vessels → Market cargoes (opportunity to earn more)")
print("    3. Market vessels → Cargill cargoes (hire at FFA rate: $18,000/day)")
print()

# Show the actual assignments from the full_result
print("📦 CARGILL CARGO COVERAGE:")
for vessel_name, cargo_name, option in full_result.cargill_vessel_assignments:
    if option.cargo_type == "cargill":
        print(f"  {vessel_name:18} → {cargo_name[:30]:32} Profit: ${option.net_profit:>10,.0f}")

for vessel_name, cargo_name, option in full_result.market_vessel_assignments:
    print(f"  {vessel_name:18} → {cargo_name[:30]:32} (Market vessel @ FFA rate)")

print()
print("💰 MARKET CARGO OPPORTUNITIES (Cargill vessels earning extra):")
for vessel_name, cargo_name, option in full_result.cargill_vessel_assignments:
    if option.cargo_type == "market":
        print(f"  {vessel_name:18} → {cargo_name[:30]:32} Profit: ${option.net_profit:>10,.0f}")

# Calculate comparison to greedy approach
print()
print("=" * 80)
print("COMPARISON: JOINT vs GREEDY OPTIMIZATION")
print("=" * 80)

# Greedy approach (old): only Cargill vessels on Cargill cargoes
greedy_portfolio = optimizer.optimize_assignments(cargill_vessels, cargill_cargoes, maximize='profit')
greedy_profit = greedy_portfolio.total_profit

# Joint approach (new): includes market cargo opportunities
joint_profit = full_result.total_profit

improvement = joint_profit - greedy_profit
improvement_pct = (improvement / greedy_profit * 100) if greedy_profit > 0 else 0

print(f"\n  Greedy (Cargill→Cargill only):  ${greedy_profit:>12,.0f}")
print(f"  Joint (+ market opportunities): ${joint_profit:>12,.0f}")
print(f"  ─────────────────────────────────────────────")
print(f"  IMPROVEMENT:                    ${improvement:>12,.0f}  (+{improvement_pct:.0f}%)")
print()
print("  Key insight: By letting Cargill vessels take lucrative market cargoes")
print("  while hiring cheaper market vessels for Cargill cargoes, we earn more overall.")

JOINT OPTIMIZATION BREAKDOWN

📊 OPTIMIZATION APPROACH:
  The joint optimizer considers ALL valid combinations simultaneously:
    1. Cargill vessels → Cargill cargoes (committed obligations)
    2. Cargill vessels → Market cargoes (opportunity to earn more)
    3. Market vessels → Cargill cargoes (hire at FFA rate: $18,000/day)

📦 CARGILL CARGO COVERAGE:
  OCEAN HORIZON      → BHP Iron Ore (Australia-China)   Profit: $   178,078
  IRON CENTURY       → EGA Bauxite (Guinea-China)       (Market vessel @ FFA rate)
  CORAL EMPEROR      → CSN Iron Ore (Brazil-China)      (Market vessel @ FFA rate)

💰 MARKET CARGO OPPORTUNITIES (Cargill vessels earning extra):
  ANN BELL           → Adaro Coal (Indonesia-India)     Profit: $ 1,146,211
  PACIFIC GLORY      → Teck Coking Coal (Canada-China   Profit: $   735,206
  GOLDEN ASCENT      → Rio Tinto Iron Ore (Australia-   Profit: $   407,376

COMPARISON: JOINT vs GREEDY OPTIMIZATION

  Greedy (Cargill→Cargill only):  $   2,283,001
  Joint (+ market o

In [102]:
# Detailed breakdown of PACIFIC VANGUARD profit on BHP cargo
print("=" * 80)
print("MARKET VESSEL PROFIT ANALYSIS: PACIFIC VANGUARD → BHP Iron Ore")
print("=" * 80)

# Get the market vessel assignment
FFA_MARKET_RATE = 18000  # $/day

for vessel_name, cargo_name, option in full_result.market_vessel_assignments:
    if option and option.result:
        result = option.result
        cargo = option.cargo  # Get cargo from the option
        
        # Calculate profit breakdown
        gross_freight = result.cargo_quantity * cargo.freight_rate
        commission = gross_freight * cargo.commission
        net_freight = result.net_freight
        
        bunker_cost = result.total_bunker_cost
        port_costs = result.port_costs
        misc_costs = result.misc_costs
        voyage_costs = bunker_cost + port_costs + misc_costs
        
        hire_cost = FFA_MARKET_RATE * result.total_days
        
        cargill_profit = net_freight - voyage_costs - hire_cost
        
        print(f"\nVessel: {vessel_name}")
        print(f"Cargo: {cargo_name}")
        print(f"\n📈 REVENUE:")
        print(f"   Cargo Quantity:     {result.cargo_quantity:>12,} MT")
        print(f"   Freight Rate:       ${cargo.freight_rate:>11.2f}/MT")
        print(f"   Gross Freight:      ${gross_freight:>12,.0f}")
        print(f"   Commission ({cargo.commission*100:.2f}%):  ${commission:>12,.0f}")
        print(f"   Net Freight:        ${net_freight:>12,.0f}")
        
        print(f"\n📉 COSTS:")
        print(f"   Bunker Cost:        ${bunker_cost:>12,.0f}")
        print(f"   Port Costs:         ${port_costs:>12,.0f}")
        print(f"   Misc Costs:         ${misc_costs:>12,.0f}")
        print(f"   ─────────────────────────────────")
        print(f"   Total Voyage Costs: ${voyage_costs:>12,.0f}")
        print(f"   Hire Cost ({result.total_days:.1f}d × ${FFA_MARKET_RATE:,}/d): ${hire_cost:>12,.0f}")
        
        print(f"\n💰 CARGILL'S PROFIT FROM THIS HIRE:")
        print(f"   Net Freight - Voyage Costs - Hire")
        print(f"   ${net_freight:,.0f} - ${voyage_costs:,.0f} - ${hire_cost:,.0f}")
        print(f"   = ${cargill_profit:,.0f}")
        
        if cargill_profit < 0:
            print(f"\n⚠️  Note: This shows a LOSS of ${abs(cargill_profit):,.0f}, but it's still optimal because:")
            print(f"   - Cargill committed to BHP cargo (MUST deliver)")
            print(f"   - ANN BELL could make it, but earns more on CSN cargo")
            print(f"   - OCEAN HORIZON could make it, but earns more on EGA cargo")
            print(f"   - Net gain: Higher profits from CSN+EGA offset this loss")
        else:
            print(f"\n✅ This hire generates positive profit for Cargill")

MARKET VESSEL PROFIT ANALYSIS: PACIFIC VANGUARD → BHP Iron Ore

Vessel: IRON CENTURY
Cargo: EGA Bauxite (Guinea-China)

📈 REVENUE:
   Cargo Quantity:          178,600 MT
   Freight Rate:       $      23.00/MT
   Gross Freight:      $   4,107,800
   Commission (1.25%):  $      51,348
   Net Freight:        $   4,056,452

📉 COSTS:
   Bunker Cost:        $   1,014,743
   Port Costs:         $           0
   Misc Costs:         $      15,000
   ─────────────────────────────────
   Total Voyage Costs: $   1,029,743
   Hire Cost (76.7d × $18,000/d): $   1,380,960

💰 CARGILL'S PROFIT FROM THIS HIRE:
   Net Freight - Voyage Costs - Hire
   $4,056,452 - $1,029,743 - $1,380,960
   = $1,645,750

✅ This hire generates positive profit for Cargill

Vessel: CORAL EMPEROR
Cargo: CSN Iron Ore (Brazil-China)

📈 REVENUE:
   Cargo Quantity:          176,950 MT
   Freight Rate:       $      22.30/MT
   Gross Freight:      $   3,945,985
   Commission (3.75%):  $     147,974
   Net Freight:        $   3,798,

In [103]:
# Show WHY joint optimization beats greedy despite any losses on market vessel hires
print("=" * 80)
print("WHY JOINT OPTIMIZATION IS OPTIMAL")
print("=" * 80)

# Option A: Greedy approach - assign Cargill vessels to Cargill cargoes only
greedy_portfolio = optimizer.optimize_assignments(cargill_vessels, cargill_cargoes, maximize='profit')
greedy_profit = greedy_portfolio.total_profit
greedy_assigned = len(greedy_portfolio.assignments)
greedy_unassigned = len(greedy_portfolio.unassigned_vessels)

print(f"\n📊 GREEDY APPROACH (Cargill vessels → Cargill cargoes only):")
print(f"   - Total profit: ${greedy_profit:,.0f}")
print(f"   - Vessels assigned: {greedy_assigned}, unassigned: {greedy_unassigned}")

# Option B: Joint optimization
print(f"\n📊 JOINT OPTIMIZATION (Current recommendation):")

# Calculate market vessel hire profit/loss dynamically
FFA_MARKET_RATE = 18000
market_hire_total = 0
market_hire_details = []

for vessel_name, cargo_name, option in full_result.market_vessel_assignments:
    if option and option.result:
        result = option.result
        cargo = option.cargo
        voyage_costs = result.total_bunker_cost + result.port_costs + result.misc_costs
        hire_cost = FFA_MARKET_RATE * result.total_days
        profit = result.net_freight - voyage_costs - hire_cost
        market_hire_total += profit
        market_hire_details.append((vessel_name, cargo_name[:28], profit))

# Sum Cargill vessel profits
cargill_vessel_profit = sum(opt.net_profit for _, _, opt in full_result.cargill_vessel_assignments)

print(f"\n   CARGILL VESSEL ASSIGNMENTS:")
for vessel_name, cargo_name, option in full_result.cargill_vessel_assignments:
    cargo_type = "Cargill" if option.cargo_type == "cargill" else "Market"
    print(f"   {vessel_name:18} → {cargo_name[:28]:30} ${option.net_profit:>12,.0f}  [{cargo_type}]")

if market_hire_details:
    print(f"\n   MARKET VESSEL HIRES:")
    for vessel_name, cargo_name, profit in market_hire_details:
        print(f"   {vessel_name:18} → {cargo_name:30} ${profit:>12,.0f}  [Hired]")

print(f"\n   ───────────────────────────────────────────────────────────────────")
print(f"   TOTAL FROM CARGILL VESSELS:                             ${cargill_vessel_profit:>12,.0f}")
print(f"   TOTAL FROM MARKET HIRES:                                ${market_hire_total:>12,.0f}")
print(f"   ═══════════════════════════════════════════════════════════════════")
net_total = cargill_vessel_profit + market_hire_total
print(f"   NET PORTFOLIO PROFIT:                                   ${net_total:>12,.0f}")

print(f"\n📈 IMPROVEMENT SUMMARY:")
improvement = net_total - greedy_profit
pct = (improvement / greedy_profit) * 100 if greedy_profit > 0 else 0
print(f"   Greedy approach:       ${greedy_profit:>12,.0f}")
print(f"   Joint optimization:    ${net_total:>12,.0f}")
print(f"   ─────────────────────────────────────")
print(f"   ADDITIONAL PROFIT:     ${improvement:>12,.0f}  (+{pct:.0f}%)")

# Show key insight based on actual data
print(f"\n💡 KEY INSIGHT:")
if market_hire_total < 0:
    print(f"   The ${abs(market_hire_total):,.0f} loss on market vessel hires is MORE than offset by:")
else:
    print(f"   Market vessel hires contributed ${market_hire_total:,.0f} plus:")
    
# Show Cargill vessels on market cargoes (the opportunity gains)
market_cargo_gains = [(v, c[:28], opt.net_profit) for v, c, opt in full_result.cargill_vessel_assignments if opt.cargo_type == "market"]
for vessel_name, cargo_name, profit in market_cargo_gains:
    print(f"   - {vessel_name} earning ${profit:,.0f} on {cargo_name}")
    
if market_cargo_gains:
    print(f"   These vessels would have been IDLE under the greedy approach!")

WHY JOINT OPTIMIZATION IS OPTIMAL

📊 GREEDY APPROACH (Cargill vessels → Cargill cargoes only):
   - Total profit: $2,283,001
   - Vessels assigned: 2, unassigned: 2

📊 JOINT OPTIMIZATION (Current recommendation):

   CARGILL VESSEL ASSIGNMENTS:
   OCEAN HORIZON      → BHP Iron Ore (Australia-Chin   $     178,078  [Cargill]
   ANN BELL           → Adaro Coal (Indonesia-India)   $   1,146,211  [Market]
   PACIFIC GLORY      → Teck Coking Coal (Canada-Chi   $     735,206  [Market]
   GOLDEN ASCENT      → Rio Tinto Iron Ore (Australi   $     407,376  [Market]

   MARKET VESSEL HIRES:
   IRON CENTURY       → EGA Bauxite (Guinea-China)     $   1,645,750  [Hired]
   CORAL EMPEROR      → CSN Iron Ore (Brazil-China)    $   1,068,424  [Hired]

   ───────────────────────────────────────────────────────────────────
   TOTAL FROM CARGILL VESSELS:                             $   2,466,871
   TOTAL FROM MARKET HIRES:                                $   2,714,174
   ════════════════════════════════════

## 9. Final Recommendation Summary

In [104]:
print("\n" + "=" * 80)
print("EXECUTIVE SUMMARY - FULL PORTFOLIO RECOMMENDATIONS")
print("=" * 80)

print("\n" + "-" * 60)
print("CARGILL VESSEL ASSIGNMENTS")
print("-" * 60)

cargill_profit = 0
for vessel_name, cargo_name, option in full_result.cargill_vessel_assignments:
    cargo_type = "Cargill" if any(cargo_name == c.name for c in cargill_cargoes) else "Market"
    print(f"\n{vessel_name} -> {cargo_name[:45]}")
    print(f"  Type: {cargo_type} cargo")
    print(f"  TCE: ${option.tce:,.0f}/day")
    print(f"  Net Profit: ${option.net_profit:,.0f}")
    print(f"  Duration: {option.voyage_days:.0f} days")
    if cargo_type == "Market":
        print(f"  Min Freight Bid: ${option.min_freight_bid:.2f}/MT")
    cargill_profit += option.net_profit

print("\n" + "-" * 60)
print("MARKET VESSEL HIRE (for Cargill cargoes)")
print("-" * 60)

if full_result.market_vessel_assignments:
    for vessel_name, cargo_name, option in full_result.market_vessel_assignments:
        print(f"\n{vessel_name} (Market) -> {cargo_name[:35]}")
        print(f"  Voyage Duration: {option.voyage_days:.1f} days")
        print(f"  Max Hire Rate: ${option.recommended_hire_rate:,.0f}/day")
else:
    print("  None required - all Cargill cargoes covered by own fleet")

print("\n" + "-" * 60)
print("UNASSIGNED ASSETS")
print("-" * 60)
if full_result.unassigned_cargill_vessels:
    print(f"  Vessels: {', '.join(full_result.unassigned_cargill_vessels)}")
else:
    print("  Vessels: All assigned")
if full_result.unassigned_cargill_cargoes:
    print(f"  Cargoes: {', '.join(full_result.unassigned_cargill_cargoes)}")
else:
    print("  Cargoes: All covered")

print("\n" + "=" * 80)
print(f"TOTAL PORTFOLIO PROFIT: ${full_result.total_profit:,.0f}")
print("=" * 80)


EXECUTIVE SUMMARY - FULL PORTFOLIO RECOMMENDATIONS

------------------------------------------------------------
CARGILL VESSEL ASSIGNMENTS
------------------------------------------------------------

OCEAN HORIZON -> BHP Iron Ore (Australia-China)
  Type: Cargill cargo
  TCE: $21,552/day
  Net Profit: $178,078
  Duration: 31 days

ANN BELL -> Adaro Coal (Indonesia-India)
  Type: Market cargo
  TCE: $29,333/day
  Net Profit: $1,146,211
  Duration: 65 days
  Min Freight Bid: $15.17/MT

PACIFIC GLORY -> Teck Coking Coal (Canada-China)
  Type: Market cargo
  TCE: $30,189/day
  Net Profit: $735,206
  Duration: 48 days
  Min Freight Bid: $15.74/MT

GOLDEN ASCENT -> Rio Tinto Iron Ore (Australia-China)
  Type: Market cargo
  TCE: $27,336/day
  Net Profit: $407,376
  Duration: 30 days
  Min Freight Bid: $9.83/MT

------------------------------------------------------------
MARKET VESSEL HIRE (for Cargill cargoes)
------------------------------------------------------------

IRON CENTURY (Ma

In [105]:
# Generate KEY INSIGHTS dynamically from computed results
print("\n" + "-" * 60)
print("KEY INSIGHTS")
print("-" * 60)

# Insight 1: Joint optimization advantage
print(f"\n1. JOINT OPTIMIZATION VALUE:")
greedy_profit = optimizer.optimize_assignments(cargill_vessels, cargill_cargoes, maximize='profit').total_profit
joint_profit = full_result.total_profit
improvement = joint_profit - greedy_profit
improvement_pct = (improvement / greedy_profit * 100) if greedy_profit > 0 else 0
print(f"   - Greedy (Cargill cargoes only): ${greedy_profit:,.0f}")
print(f"   - Joint optimization: ${joint_profit:,.0f}")
print(f"   - Additional profit: ${improvement:,.0f} (+{improvement_pct:.0f}%)")

# Insight 2: Vessel utilization
print(f"\n2. VESSEL UTILIZATION:")
n_cargill_on_cargill = sum(1 for _, _, opt in full_result.cargill_vessel_assignments if opt.cargo_type == "cargill")
n_cargill_on_market = sum(1 for _, _, opt in full_result.cargill_vessel_assignments if opt.cargo_type == "market")
n_market_hired = len(full_result.market_vessel_assignments)
print(f"   - Cargill vessels on Cargill cargoes: {n_cargill_on_cargill}")
print(f"   - Cargill vessels on Market cargoes: {n_cargill_on_market}")
print(f"   - Market vessels hired for Cargill cargoes: {n_market_hired}")
print(f"   - Unassigned Cargill vessels: {len(full_result.unassigned_cargill_vessels)}")

# Insight 3: Market cargo opportunities
print(f"\n3. MARKET CARGO OPPORTUNITIES:")
for vessel_name, cargo_name, option in full_result.cargill_vessel_assignments:
    if option.cargo_type == "market":
        print(f"   - {vessel_name} on {cargo_name[:30]}")
        print(f"     TCE: ${option.tce:,.0f}/day, Min Bid: ${option.min_freight_rate:.2f}/MT")

# Insight 4: Market vessel hire
print(f"\n4. MARKET VESSEL HIRE:")
if full_result.market_vessel_assignments:
    for vessel_name, cargo_name, option in full_result.market_vessel_assignments:
        print(f"   - {vessel_name} hired for {cargo_name[:30]}")
        print(f"     Duration: {option.voyage_days:.0f} days at FFA rate ($18,000/day)")
else:
    print("   - No market vessels needed")

# Insight 5: Scenario robustness (from earlier analysis)
print(f"\n5. SCENARIO ROBUSTNESS:")
if bunker_tipping_pct:
    print(f"   - Recommendation stable until bunker prices increase >{bunker_tipping_pct:.0f}%")
else:
    print(f"   - Recommendation robust to bunker price increases up to +{max_bunker:.0f}%")
if delay_tipping_days:
    print(f"   - Recommendation stable until port delays exceed {delay_tipping_days} days")
else:
    print(f"   - Recommendation robust to port delays up to {max_delay} days")


------------------------------------------------------------
KEY INSIGHTS
------------------------------------------------------------

1. JOINT OPTIMIZATION VALUE:
   - Greedy (Cargill cargoes only): $2,283,001
   - Joint optimization: $5,181,045
   - Additional profit: $2,898,044 (+127%)

2. VESSEL UTILIZATION:
   - Cargill vessels on Cargill cargoes: 1
   - Cargill vessels on Market cargoes: 3
   - Market vessels hired for Cargill cargoes: 2
   - Unassigned Cargill vessels: 0

3. MARKET CARGO OPPORTUNITIES:
   - ANN BELL on Adaro Coal (Indonesia-India)
     TCE: $29,333/day, Min Bid: $15.17/MT
   - PACIFIC GLORY on Teck Coking Coal (Canada-China
     TCE: $30,189/day, Min Bid: $15.74/MT
   - GOLDEN ASCENT on Rio Tinto Iron Ore (Australia-
     TCE: $27,336/day, Min Bid: $9.83/MT

4. MARKET VESSEL HIRE:
   - IRON CENTURY hired for EGA Bauxite (Guinea-China)
     Duration: 77 days at FFA rate ($18,000/day)
   - CORAL EMPEROR hired for CSN Iron Ore (Brazil-China)
     Duration: 77 day

In [106]:
print("\n" + "-" * 60)
print("RISK FACTORS")
print("-" * 60)
print("""
- Weather: Long-haul routes (Brazil, Guinea) exposed to weather delays
- Congestion: Chinese ports showing increased waiting times
- Bunker: Prices volatile due to geopolitical factors
- BHP Laycan: Very tight window (Mar 7-11), limited market vessel options
""")


------------------------------------------------------------
RISK FACTORS
------------------------------------------------------------

- Weather: Long-haul routes (Brazil, Guinea) exposed to weather delays
- Congestion: Chinese ports showing increased waiting times
- Bunker: Prices volatile due to geopolitical factors
- BHP Laycan: Very tight window (Mar 7-11), limited market vessel options



---

## Appendix: Key Assumptions

| Parameter | Assumption | Source |
|-----------|------------|--------|
| Speed Mode | Economical (lower fuel consumption) | Industry standard for cost optimization |
| Bunker Prices | March 2026 forward curve | Datathon provided data |
| Port Working Hours | PWWD SHINC basis | Cargo terms |
| Weather Allowance | Not included in base case | Scenario analysis covers delays |
| Misc Costs | $15,000 per voyage | Industry estimate |
| Cargo Tolerance | Maximum within vessel DWT | Owners' option exercised |
| FFA Market Rate (Hire) | $18,000/day | 5TC March 2026 reference |
| Target TCE (market bids) | $18,000/day | FFA market reference |

## Optimization Approach: Joint vs Greedy

| Approach | Method | Result |
|----------|--------|--------|
| **Greedy** | Assign Cargill vessels to Cargill cargoes only | ~$2.3M profit |
| **Joint** | Optimize across all valid combinations | ~$4.8M profit (+109%) |

**Key insight:** By allowing Cargill vessels to take lucrative market cargoes while hiring market vessels at FFA rate for committed Cargill cargoes, total portfolio profit increases significantly.

## Valid Optimization Combinations

| Vessel Type | Cargo Type | Allowed | Purpose |
|-------------|------------|---------|---------|
| Cargill | Cargill | ✅ Yes | Fulfill committed obligations |
| Cargill | Market | ✅ Yes | Bid for additional revenue |
| Market | Cargill | ✅ Yes | Hire to cover commitments |
| Market | Market | ❌ No | Not Cargill's business |

---

*Report generated using Python-based Freight Calculator and Joint Portfolio Optimizer*